<a href="https://colab.research.google.com/github/guebin/DL2024/blob/main/posts/08wk-1-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상 

In [1]:
#{{<video https://youtu.be/playlist?list=PLQqh36zP38-xfh-AQQI0B_GONOjgj9DCi&si=uWImDc1bYBoNqCB_ >}}

# 2. Imports 

In [2]:
import torch
import numpy as np 
import pandas as pd
import fastai.collab

# 3. 예비학습: optimizer 사용 고급 

# 4. 나는 SOLO 최초구현

ref: <https://namu.wiki/w/나는%20SOLO>

## A. Data

`-` Data 

In [331]:
!wget https://raw.githubusercontent.com/guebin/STML2022/main/posts/V.%20RecSys/2022-12-21-rcmdsolo.csv

--2024-05-05 03:38:09--  https://raw.githubusercontent.com/guebin/STML2022/main/posts/V.%20RecSys/2022-12-21-rcmdsolo.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 193 [text/plain]
Saving to: ‘2022-12-21-rcmdsolo.csv’

2022-12-21-rcmdsolo 100%[===================>]     193  --.-KB/s    in 0s      

2024-05-05 03:38:09 (6.99 MB/s) - ‘2022-12-21-rcmdsolo.csv’ saved [193/193]



In [594]:
df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/STML2022/main/posts/V.%20RecSys/2022-12-21-rcmdsolo.csv',index_col=0)
df_view 

,영식,영철,영호,광수,상철,영수
옥순,3.9,4.1,NaN,0.5,0.3,NaN
영자,4.5,NaN,3.7,0.5,NaN,0.2
정숙,NaN,4.9,4.7,NaN,1.2,1.3
영숙,0.6,0.2,NaN,4.1,4.3,NaN
순자,0.7,0.9,NaN,4.2,NaN,3.9
현숙,NaN,0.2,0.3,NaN,3.5,3.4


In [639]:
df_view.iloc[-2,-1] = 4.8

In [641]:
df_view

,영식,영철,영호,광수,상철,영수
옥순,3.9,4.1,NaN,0.5,0.3,NaN
영자,4.5,NaN,3.7,0.5,NaN,0.4
정숙,NaN,4.9,4.7,NaN,1.2,1.3
영숙,0.6,0.2,NaN,4.1,4.3,NaN
순자,0.7,0.9,NaN,4.2,NaN,4.8
현숙,NaN,0.2,0.3,NaN,3.5,3.4


In [642]:
df_view.iloc[1,-1] = 0.4

`-` 데이터를 이해할 때 필요한 가정들 -- 제가 마음대로 설정했어요.. 

- 궁합이 잘맞으면 5점, 잘 안맞으면 0점 이다. 
- ((옥순,영자,정숙),(영식,영철,영호))은 MBTI가 I로 시작하고 ((영숙,순자,현숙),(광수,상철,영수))는 MBTI가 E로 시작한다고 설정하자.
- (옥순,영자,정숙)은 (영식,영철,영호)와 성격이 잘 맞고 (영숙,순자,현숙)은 (광수,상철,영수)와 성격이 잘맞음. 
- 정숙은 대체적으로 모든 사람들이랑 궁합이 잘 맞는 편인것 같다. 
- 현숙은 전체적으로 모든 사람들이랑 궁합이 잘 안맞는 편인것 같다. (눈이 높아보인다)

## B. Fit / Predict

`-` 목표: NaN을 추정 

`-` 수동추론: 

- (옥순,영호)이 만난다면? $\to$ 둘다 I성향이니까 잘 맞지 않을까? $\to$ 4.0 정도? 
- (정숙,영식)조합은? $\to$ 둘다 I성향이니까 잘 맞지 않을까? + 정숙은 다 잘맞던데..? $\to$ 4.8 정도? 
- (현숙,영식)조합은? $\to$ 현숙은 E성향인데 영식은 I성향이므로 잘 안맞을 것임 + 현숙은 원래 좀 눈이 높음 $\to$ 0.25 정도? 

`-` 좀 더 체계적인 추론 전략: 사람들이 가지고 있는 성향들을 두 개의 숫자로 표현하자. 

- 옥순의 성향 = (I성향,E성향) = (1.9, 0.0) 
- 영식의 성향 = (I성향,E성향) = (2.0, 0.1) 
- 현숙의 성향 = (I성향,E성향) = (0.0, 1.5) 

> 사람에 따라 E,I 둘다 높게 설정할수도있고, 둘다 낮게 설정할 수도 있다. 

**(1)** 옥순과 영식의 궁합 $\approx$ 옥순의I성향$\times$영식의I성향 $+$ 옥순의E성향$\times$영식의E성향 // 적합

In [643]:
옥순성향 = torch.tensor([1.9,0.0]).reshape(2,1) 
영식성향 = torch.tensor([2.0,0.1]).reshape(2,1) 
#(옥순성향*영식성향).sum().item() # 옥순과 영식의 궁합: a ∘ b 로 내적구함 
(옥순성향.T @ 영식성향).item() # 옥순과 영식의 궁합: a.T @ b 로 내적구함 

3.799999952316284

**(2)** 현숙과 영식의 궁합 $\approx$ 현숙의I성향$\times$영식의I성향 $+$ 현숙의E성향$\times$영식의E성향 // 예측

In [644]:
현숙성향 = torch.tensor([0.0,1.5]).reshape(2,1)
#(현숙성향*영식성향).sum().item() # 현숙과 영식의 궁합: a ∘ b 로 내적구함 
(현숙성향.T @ 영식성향).item() # 현숙과 영식의 궁합: a.T @ b 로 내적구함 

0.15000000596046448

- 그럴싸함. 

`-` 전체 사용자의 설정값 

In [645]:
옥순성향 = a1 = torch.tensor([1.9,0.0]).reshape(2,1)
영자성향 = a2 = torch.tensor([2.0,0.1]).reshape(2,1)
정숙성향 = a3 = torch.tensor([2.5,1.0]).reshape(2,1)
영숙성향 = a4 = torch.tensor([0.1,1.9]).reshape(2,1)
순자성향 = a5 = torch.tensor([0.2,2.1]).reshape(2,1)
현숙성향 = a6 = torch.tensor([0.0,1.5]).reshape(2,1)
A = torch.concat([a1,a2,a3,a4,a5,a6],axis=1)
A # 각 column은 여성출연자들의 성향을 의미함 

tensor([[1.9000, 2.0000, 2.5000, 0.1000, 0.2000, 0.0000],
        [0.0000, 0.1000, 1.0000, 1.9000, 2.1000, 1.5000]])

In [646]:
영식성향 = b1 = torch.tensor([2.0,0.1]).reshape(2,1)
영철성향 = b2 = torch.tensor([1.9,0.2]).reshape(2,1)
영호성향 = b3 = torch.tensor([1.8,0.3]).reshape(2,1)
광수성향 = b4 = torch.tensor([0.3,2.1]).reshape(2,1)
상철성향 = b5 = torch.tensor([0.2,2.0]).reshape(2,1)
영수성향 = b6 = torch.tensor([0.1,1.9]).reshape(2,1)
B = torch.concat([b1,b2,b3,b4,b5,b6],axis=1)
B # 각 column은 남성출연자의 성향을 의미함

tensor([[2.0000, 1.9000, 1.8000, 0.3000, 0.2000, 0.1000],
        [0.1000, 0.2000, 0.3000, 2.1000, 2.0000, 1.9000]])

`-` 아래의 행렬곱 관찰 

In [647]:
A.T@B 

tensor([[3.8000, 3.6100, 3.4200, 0.5700, 0.3800, 0.1900],
        [4.0100, 3.8200, 3.6300, 0.8100, 0.6000, 0.3900],
        [5.1000, 4.9500, 4.8000, 2.8500, 2.5000, 2.1500],
        [0.3900, 0.5700, 0.7500, 4.0200, 3.8200, 3.6200],
        [0.6100, 0.8000, 0.9900, 4.4700, 4.2400, 4.0100],
        [0.1500, 0.3000, 0.4500, 3.1500, 3.0000, 2.8500]])

---저거 따져보자--- 

${\bf A}_{2 \times 6} = \begin{bmatrix} {\boldsymbol a}_{\text{옥순}} & {\boldsymbol a}_{\text{영자}}  & {\boldsymbol a}_{\text{정숙}} & {\boldsymbol a}_{\text{영숙}}  & {\boldsymbol a}_{\text{순자}}  & {\boldsymbol a}_{\text{현숙}} \end{bmatrix} = \begin{bmatrix} 1.9 & 2.0 & 2.5 & 0.1 & 0.2  & 0 \\  0 & 0.1 & 1.0 & 1.9 & 2.1  & 1.5  \end{bmatrix}$

${\bf B}_{2 \times 6} = \begin{bmatrix} {\boldsymbol b}_{\text{영식}} & {\boldsymbol b}_{\text{영철}}  & {\boldsymbol b}_{\text{영호}} & {\boldsymbol b}_{\text{광수}}  & {\boldsymbol b}_{\text{상철}}  & {\boldsymbol b}_{\text{영수}} \end{bmatrix} = \begin{bmatrix} 2.0 & 1.0 & 1.8 & 0.3 & 0.2 & 0.1 \\  0.1 & 0.2 & 0.3 & 2.1 & 2.0 & 1.9  \end{bmatrix}$

$\begin{align*}{\bf A}^\top @ {\bf B} & = \begin{bmatrix}
1.9 & 0.0 \\
2.0 & 0.1 \\
2.5 & 1.0 \\
0.1 & 1.9 \\
0.2 & 2.1 \\
0.0 & 1.5 \\
\end{bmatrix} \begin{bmatrix} 2.0 & 1.0 & 1.8 & 0.3 & 0.2 & 0.1 \\  0.1 & 0.2 & 0.3 & 2.1 & 2.0 & 1.9  \end{bmatrix} \\ \\ 
& = \begin{bmatrix}
3.8 & 3.61 & 3.42 & 0.57 & 0.38 & 0.19 \\
4.01 & 3.82 & 3.63 & 0.81 & 0.6 & 0.39 \\
5.1 & 4.95 & 4.8 & 2.85 & 2.5 & 2.15 \\
0.39 & 0.57 & 0.75 & 4.02 & 3.82 & 3.62 \\
0.61 & 0.8 & 0.99 & 4.47 & 4.24 & 4.01 \\
0.15 & 0.3 & 0.45 & 3.15 & 3 & 2.85 \\
\end{bmatrix} \\ \\ 
& =\begin{bmatrix}
{\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영수}} \\
{\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영수}}  \\
{\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영수}} \\
{\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영수}}  \\
{\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영수}}  \\
{\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영수}}  \\
\end{bmatrix}\end{align*}$

`-` 궁합매트릭스: ${\bf A}^\top @ {\bf B}$를 계산하면 (6,6) 인 행렬이 나올텐데 이 행렬의 $(i,j)$의 원소는 $i$-th 여성출연자와 $j$-th 남성출연자가 얼마나 잘 맞는지를 나타내는 숫자가 된다. (숫자가 높을수록 잘 맞음) 이 수업에서는 이 매트릭스를 "궁합매트릭스" 라고 정의하자. 

- 걱정1: 5.0이 넘는 값도 있네?
- 걱정2: 잘못잡으면 음수가 나올지도? 
- 로지스틱 느낌 나지 않음??

`-` 주어진 자료와 우리가 임의로 만든 궁합매트릭스를 비교해보자. 

In [648]:
A.T @ B, torch.tensor(np.array(df_view)).float()

(tensor([[3.8000, 3.6100, 3.4200, 0.5700, 0.3800, 0.1900],
         [4.0100, 3.8200, 3.6300, 0.8100, 0.6000, 0.3900],
         [5.1000, 4.9500, 4.8000, 2.8500, 2.5000, 2.1500],
         [0.3900, 0.5700, 0.7500, 4.0200, 3.8200, 3.6200],
         [0.6100, 0.8000, 0.9900, 4.4700, 4.2400, 4.0100],
         [0.1500, 0.3000, 0.4500, 3.1500, 3.0000, 2.8500]]),
 tensor([[3.9000, 4.1000,    nan, 0.5000, 0.3000,    nan],
         [4.5000,    nan, 3.7000, 0.5000,    nan, 0.4000],
         [   nan, 4.9000, 4.7000,    nan, 1.2000, 1.3000],
         [0.6000, 0.2000,    nan, 4.1000, 4.3000,    nan],
         [0.7000, 0.9000,    nan, 4.2000,    nan, 4.8000],
         [   nan, 0.2000, 0.3000,    nan, 3.5000, 3.4000]]))

`-` 우리의 전략

1. ${\bf A}^\top @~ {\bf B}$의 값과 `df_view` 의 값이 `nan`을 제외한 곳에서 거의 비슷하게 되도록 ${\bf A}$, ${\bf B}$를 잘 때려맞추면 되는것 아니야?

2. 1을 만족하는 ${\bf A}$, ${\bf B}$를 찾았으면 그 숫자들을 이용하여 `df_view`의 `nan` 을 추정한다. 

`-` 따라서 모형은 아래와 같이 볼 수 있다. 

$${\tt df\_view} \approx {\bf A}^\top @~ {\bf B}$$

`-` 아래의 정보를 참고하여 위의 수식을 다시 정리하면..

In [649]:
df_view

,영식,영철,영호,광수,상철,영수
옥순,3.9,4.1,NaN,0.5,0.3,NaN
영자,4.5,NaN,3.7,0.5,NaN,0.4
정숙,NaN,4.9,4.7,NaN,1.2,1.3
영숙,0.6,0.2,NaN,4.1,4.3,NaN
순자,0.7,0.9,NaN,4.2,NaN,4.8
현숙,NaN,0.2,0.3,NaN,3.5,3.4


In [650]:
display(pd.DataFrame(A.T.tolist()).set_axis(['옥순','영자','정숙','영숙','순자','현숙']).set_axis(['I성향','E성향'],axis=1))
display(pd.DataFrame(B.T.tolist()).set_axis(['영식','영철','영호','광수','상철','영수']).set_axis(['I성향','E성향'],axis=1))

,I성향,E성향
옥순,1.9,0.0
영자,2.0,0.1
정숙,2.5,1.0
영숙,0.1,1.9
순자,0.2,2.1
현숙,0.0,1.5


,I성향,E성향
영식,2.0,0.1
영철,1.9,0.2
영호,1.8,0.3
광수,0.3,2.1
상철,0.2,2.0
영수,0.1,1.9


---이런식으로?---

|W|M|W_IE := ${\boldsymbol a}_{W}^T$|M_IE := ${\boldsymbol b}_{M}^T$|yhat|y| 
|:-:|:-:|:-:|:-:|:-:|:-:
|옥순|영식|[1.9, 0.0] | [2.0, 0.1] |3.8| 3.9|
|옥순|영철|[1.9, 0.0] | [1.9, 0.2] |3.6| 4.1|
|옥순|광수|[1.9, 0.0] | [0.3, 2.1] |0.57| 0.5|
|옥순|상철|[1.9, 0.0] | [0.2, 2.0] |0.38| 0.3|
|...|...|... | ... | ...|...|
|현숙|영철|[0.0, 1.5] | [1.9, 0.2] |0.3| 0.2|
|현숙|영호|[0.0, 1.5] | [1.8, 0.3] |0.45| 0.2|
|현숙|상철|[0.0, 1.5] | [0.2, 2.0] |3.0| 0.2|
|현숙|영수|[0.0, 1.5] | [0.1, 1.9] |2.85| 0.2|

`-` 이렇게 정리하면 `yhat` $\approx$ `y` 를 만족하도록 하면 된다. 

- `yhat`은 어떻게 구하지? `(W_IE * M_IE).sum()`? 
- 그럼 `W_IE`,`M_IE`는 어떻게 구하지?? 생각해보니까 데이터에서 주어진건 아니잖아

`-` `W_IE`,`M_IE` 를 어떻게 만들지?

- 그전엔 어떻게 했지?? `W`을 보고 적당히 특징을 상상하고 `W_IE`를 때려넣음 + `M`를 보고 적당히 특징을 상상하고 `M_IE`를 넣음. 
- 자동화하려면? `W` $\to$ `W_IE` 인 함수를 만들고 `M` $\to$ `M_IE` 인 함수를 만들자.

`-` 앞으로 할일1:  `W` $\to$ `W_IE` 인 함수를 만들고 `M` $\to$ `M_IE` 인 함수를 만들자. 즉 
    
- 옥순 $\to$ 1.9, 0.0
- 영자 $\to$ 2.0, 0.1 
...
- 현숙 $\to$ 0.0, 1.5 

로 가는 함수를 하나 구현하고 

- 영식 $\to$ 2.0, 0.1
- 영철 $\to$ 1.9, 0.2 
...
- 영수 $\to$ 0.1, 1.9 

를 와 같은 함수도 하나 구현한다. 

`-` 앞으로 할일2: 우리가 익숙한 셋팅 (step1~4)
    
1. `W_IE`, `M_IE` $\to$ `yhat` 를 수행
2. `y` $\approx$ `yhat` 인지 체크: `loss = loss_fn(yhat,y)`
3. `loss.backward()`
4. 더 나은 `W_IE`, `M_IE` 가 생기도록 update 

## C. 할일1: `W` $\to$ `W_IE`, `M` $\to$ `M_IE` 의 구현

`-` dataframe의 변형 

In [651]:
df = df_view.stack().reset_index().set_axis(['W','M','y'],axis=1)
df[:5]

,W,M,y
0,옥순,영식,3.9
1,옥순,영철,4.1
2,옥순,광수,0.5
3,옥순,상철,0.3
4,영자,영식,4.5


`-` 이름을 숫자화

In [652]:
w = {'옥순':0, '영자':1, '정숙':2, '영숙':3, '순자':4, '현숙':5}
m = {'영식':0, '영철':1, '영호':2, '광수':3, '상철':4, '영수':5}

In [653]:
df['X1'] = df['W'].map(w)
df['X2'] = df['M'].map(m)
df[:5]

,W,M,y,X1,X2
0,옥순,영식,3.9,0,0
1,옥순,영철,4.1,0,1
2,옥순,광수,0.5,0,3
3,옥순,상철,0.3,0,4
4,영자,영식,4.5,1,0


`-` 텐서화 + one_hot-인코딩 

In [654]:
y = torch.tensor(df['y']).float().reshape(-1,1)
E1 = torch.nn.functional.one_hot(torch.tensor(df['X1'])).float()
E2 = torch.nn.functional.one_hot(torch.tensor(df['X2'])).float()
print(f'y.shape: {y.shape}')
print(f'E1.shape: {E1.shape} // 이때 {E1.shape[-1]}은 여성참가자의 수')
print(f'E2.shape: {E2.shape} // 이때 {E2.shape[-1]}은 남성참가자의 수')

y.shape: torch.Size([24, 1])
E1.shape: torch.Size([24, 6]) // 이때 6은 여성참가자의 수
E2.shape: torch.Size([24, 6]) // 이때 6은 남성참가자의 수


`-` `X1` -> `X1_IE`, `X2` -> `X2_IE` 인 변환구현  

In [655]:
torch.manual_seed(43052)
l1 = torch.nn.Linear(in_features=6, out_features=2)
l2 = torch.nn.Linear(in_features=6, out_features=2)
W_IE = l1(E1) # W_IE = l1(onehot(X1))
M_IE = l2(E2) # M_IE = l2(onehot(X2))

In [656]:
W_IE[::5], M_IE[::5]

(tensor([[-0.4139,  0.2151],
         [-0.6182, -0.1716],
         [-0.1252, -0.0238],
         [ 0.1087,  0.3370],
         [-0.6351, -0.1291]], grad_fn=<SliceBackward0>),
 tensor([[ 0.5298,  0.1328],
         [ 0.6899, -0.1798],
         [ 0.3161,  0.3028],
         [ 0.3161,  0.3028],
         [ 0.2741,  0.3177]], grad_fn=<SliceBackward0>))

## D. 할일2: step1~4 수행 

`-` step1: yhat을 구하자. 

In [657]:
W_IE[::5] # 여성참가자의 성향들 

tensor([[-0.4139,  0.2151],
        [-0.6182, -0.1716],
        [-0.1252, -0.0238],
        [ 0.1087,  0.3370],
        [-0.6351, -0.1291]], grad_fn=<SliceBackward0>)

In [658]:
M_IE[::5] # 남성참가자의 성향들

tensor([[ 0.5298,  0.1328],
        [ 0.6899, -0.1798],
        [ 0.3161,  0.3028],
        [ 0.3161,  0.3028],
        [ 0.2741,  0.3177]], grad_fn=<SliceBackward0>)

In [659]:
(-0.4139 * 0.5298) + (0.2151 * 0.1328) # 궁합을 의미하는 값 

-0.19071894

In [660]:
(W_IE * M_IE).sum(axis=1).reshape(-1,1)[::5]

tensor([[-0.1907],
        [-0.3957],
        [-0.0468],
        [ 0.1364],
        [-0.2151]], grad_fn=<SliceBackward0>)

출력결과를 0~5사이로 맞춰주자.

In [661]:
sig = torch.nn.Sigmoid()
yhat = sig((W_IE * M_IE).sum(axis=1).reshape(-1,1))*5
yhat

tensor([[2.2623],
        [2.4436],
        [2.5254],
        [2.4179],
        [2.0665],
        [2.0118],
        [2.2822],
        [1.9860],
        [2.4477],
        [2.3974],
        [2.4415],
        [2.3926],
        [2.6278],
        [2.6709],
        [2.6975],
        [2.6703],
        [2.5091],
        [2.6533],
        [2.7257],
        [2.3260],
        [2.2321],
        [1.9886],
        [2.2016],
        [1.9648]], grad_fn=<MulBackward0>)

`-` step2: 손실계산 

In [662]:
loss_fn = torch.nn.MSELoss()
loss = loss_fn(yhat,y)

`-` step3: 미분 

In [663]:
l1.weight.data,l1.bias.data,l2.weight.data,l2.bias.data

(tensor([[-0.1415, -0.3458,  0.1472,  0.3811,  0.1663, -0.3627],
         [ 0.1330, -0.2537, -0.1058,  0.2549,  0.3958, -0.2112]]),
 tensor([-0.2724,  0.0821]),
 tensor([[ 0.1881, -0.0676,  0.3482, -0.1746, -0.0257,  0.3702],
         [-0.0318,  0.1532, -0.3444,  0.2514,  0.1382, -0.2983]]),
 tensor([0.3417, 0.1646]))

In [664]:
l1.weight.grad,l1.bias.grad,l2.weight.grad,l2.bias.grad 

(None, None, None, None)

In [665]:
loss.backward()

In [666]:
l1.weight.data,l1.bias.data,l2.weight.data,l2.bias.data

(tensor([[-0.1415, -0.3458,  0.1472,  0.3811,  0.1663, -0.3627],
         [ 0.1330, -0.2537, -0.1058,  0.2549,  0.3958, -0.2112]]),
 tensor([-0.2724,  0.0821]),
 tensor([[ 0.1881, -0.0676,  0.3482, -0.1746, -0.0257,  0.3702],
         [-0.0318,  0.1532, -0.3444,  0.2514,  0.1382, -0.2983]]),
 tensor([0.3417, 0.1646]))

In [667]:
l1.weight.grad,l1.bias.grad,l2.weight.grad,l2.bias.grad 

(tensor([[-0.0319, -0.1035, -0.1135,  0.1042, -0.0583,  0.0300],
         [ 0.0772,  0.0532, -0.0141, -0.0022,  0.0538,  0.0149]]),
 tensor([-0.1731,  0.1828]),
 tensor([[ 0.2249, -0.0210,  0.0275, -0.2008, -0.0411,  0.0058],
         [ 0.1669,  0.1152,  0.0130, -0.1080,  0.0046, -0.1340]]),
 tensor([-0.0048,  0.0577]))

`-` step4: update 

In [668]:
optimizr = torch.optim.Adam(list(l1.parameters())+list(l2.parameters()))
optimizr.step()

In [669]:
l1.weight.data,l1.bias.data,l2.weight.data,l2.bias.data

(tensor([[-0.1405, -0.3448,  0.1482,  0.3801,  0.1673, -0.3637],
         [ 0.1320, -0.2547, -0.1048,  0.2559,  0.3948, -0.2122]]),
 tensor([-0.2714,  0.0811]),
 tensor([[ 0.1871, -0.0666,  0.3472, -0.1736, -0.0247,  0.3692],
         [-0.0328,  0.1522, -0.3454,  0.2524,  0.1372, -0.2973]]),
 tensor([0.3427, 0.1636]))

In [670]:
l1.weight.grad,l1.bias.grad,l2.weight.grad,l2.bias.grad 

(tensor([[-0.0319, -0.1035, -0.1135,  0.1042, -0.0583,  0.0300],
         [ 0.0772,  0.0532, -0.0141, -0.0022,  0.0538,  0.0149]]),
 tensor([-0.1731,  0.1828]),
 tensor([[ 0.2249, -0.0210,  0.0275, -0.2008, -0.0411,  0.0058],
         [ 0.1669,  0.1152,  0.0130, -0.1080,  0.0046, -0.1340]]),
 tensor([-0.0048,  0.0577]))

In [671]:
optimizr.zero_grad()

In [672]:
l1.weight.data,l1.bias.data,l2.weight.data,l2.bias.data

(tensor([[-0.1405, -0.3448,  0.1482,  0.3801,  0.1673, -0.3637],
         [ 0.1320, -0.2547, -0.1048,  0.2559,  0.3948, -0.2122]]),
 tensor([-0.2714,  0.0811]),
 tensor([[ 0.1871, -0.0666,  0.3472, -0.1736, -0.0247,  0.3692],
         [-0.0328,  0.1522, -0.3454,  0.2524,  0.1372, -0.2973]]),
 tensor([0.3427, 0.1636]))

In [673]:
l1.weight.grad,l1.bias.grad,l2.weight.grad,l2.bias.grad 

(None, None, None, None)

## E. 정리된 최종코드 

`-` 데이터를 불러오기 

In [674]:
#df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/STML2022/main/posts/V.%20RecSys/2022-12-21-rcmdsolo.csv',index_col=0)
df = df_view.stack().reset_index().set_axis(['W','M','y'],axis=1)
w = {'옥순':0, '영자':1, '정숙':2, '영숙':3, '순자':4, '현숙':5}
m = {'영식':0, '영철':1, '영호':2, '광수':3, '상철':4, '영수':5}
X1 = torch.tensor(df['W'].map(w)) # length-n vector 
X2 = torch.tensor(df['M'].map(m)) # length-n vector 
E1 = torch.nn.functional.one_hot(X1).float()
E2 = torch.nn.functional.one_hot(X2).float()
y = torch.tensor(df['y']).float().reshape(-1,1)

`-` 데이터의 차원 및 dtype가 헷갈릴수 있는데 아래와 같이 되어있어야한다. 

In [675]:
print(f"X1: shape = {X1.shape}, dtype = {X1.dtype}") # one-hot 인코딩 넣을거라서.. lenght-n 벡터로
print(f"X2: shape = {X2.shape}, dtype = {X2.dtype}") # one-hot 인코딩 넣을거라서.. lenght-n 벡터로
print(f"E1: shape = {E1.shape}, dtype = {E1.dtype}") # one-hot 인코딩 처리된 X1
print(f"E2: shape = {E2.shape}, dtype = {E2.dtype}") # one-hot 인코딩 처리된 X2
print(f"y: shape = {y.shape}, dtype = {y.dtype}")

X1: shape = torch.Size([24]), dtype = torch.int64
X2: shape = torch.Size([24]), dtype = torch.int64
E1: shape = torch.Size([24, 6]), dtype = torch.float32
E2: shape = torch.Size([24, 6]), dtype = torch.float32
y: shape = torch.Size([24, 1]), dtype = torch.float32


`-` 학습

In [676]:
# 
torch.manual_seed(43052)
l1 = torch.nn.Linear(in_features=6,out_features=2)
l2 = torch.nn.Linear(in_features=6,out_features=2)
sig = torch.nn.Sigmoid()
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(
    list(l1.parameters())+list(l2.parameters()),
    lr = 0.01
)
#---#
for epoc in range(300):
    ## step1 
    W_features = l1(E1) # W_features = l1(E1)
    M_features = l2(E2) # M_features = l1(E1)
    dot = (W_features * M_features).sum(axis=1)
    yhat = (sig(dot)*5).reshape(-1,1)
    ## step2 
    loss = loss_fn(yhat,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()

`-` 학습된결과 

In [677]:
df_ebdspace = pd.DataFrame(torch.concat([W_features.data, M_features.data],axis=1))
df_ebdspace.columns = ['여성잠재특징1','여성잠재특징2','남성잠재특징1','남성잠재특징2']
df_yhat = pd.DataFrame(yhat.data,columns=['yhat'])

In [678]:
pd.concat([df,df_yhat,df_ebdspace],axis=1)

,W,M,y,yhat,여성잠재특징1,여성잠재특징2,남성잠재특징1,남성잠재특징2
0,옥순,영식,3.9,4.143693,0.267892,-1.397888,0.750719,-0.984057
1,옥순,영철,4.1,4.013299,0.267892,-1.397888,2.178255,-0.586216
2,옥순,광수,0.5,0.560297,0.267892,-1.397888,0.911421,1.655382
3,옥순,상철,0.3,0.458212,0.267892,-1.397888,0.124914,1.664804
4,영자,영식,4.5,4.299052,0.234309,-1.664350,0.750719,-0.984057
5,영자,영호,3.7,3.760194,0.234309,-1.664350,1.862390,-0.404446
6,영자,광수,0.5,0.364971,0.234309,-1.664350,0.911421,1.655382
7,영자,영수,0.4,0.346665,0.234309,-1.664350,0.128163,1.578400
8,정숙,영철,4.9,4.823182,1.307826,-0.780071,2.178255,-0.586216
9,정숙,영호,4.7,4.699900,1.307826,-0.780071,1.862390,-0.404446


## F. 예측? 

`-` "(옥순,영호)" 는 잘 맞을까? --> 그렇지 않을까?

In [630]:
df_view

,영식,영철,영호,광수,상철,영수
옥순,3.9,4.1,NaN,0.5,0.3,NaN
영자,4.5,NaN,3.7,0.5,NaN,0.2
정숙,NaN,4.9,4.7,NaN,1.2,1.3
영숙,0.6,0.2,NaN,4.1,4.3,NaN
순자,0.7,0.9,NaN,4.2,NaN,3.9
현숙,NaN,0.2,0.3,NaN,3.5,3.4


In [631]:
E1

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.]])

In [632]:
E1[[0],:] # 옥순
E2[[2],:] # 영호 

tensor([[0., 0., 0., 1., 0., 0.]])

In [633]:
l1(E1[[0],:]).data, l2(E2[[2],:]).data

(tensor([[-0.4119,  0.2131]]), tensor([[0.1692, 0.4159]]))

# 5. 나는 SOLO 다른구현 -- bias를 고려 

In [681]:
# 
torch.manual_seed(43052)
l1 = torch.nn.Linear(in_features=6,out_features=2)
b1 = torch.nn.Linear(in_features=6,out_features=1)
l2 = torch.nn.Linear(in_features=6,out_features=2)
b2 = torch.nn.Linear(in_features=6,out_features=1)
sig = torch.nn.Sigmoid()
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(
    list(l1.parameters())+list(l2.parameters())+list(b1.parameters())+list(b2.parameters()),
    lr = 0.01
)
#---#
for epoc in range(300):
    ## step1 
    W_features = l1(E1) 
    W_bias = b1(E1)
    M_features = l2(E2) 
    M_bias = b2(E2)
    dot = (W_features * M_features + W_bias + M_bias).sum(axis=1)
    yhat = (sig(dot)*5).reshape(-1,1)
    ## step2 
    loss = loss_fn(yhat,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()

In [682]:
df_ebdspace = pd.DataFrame(torch.concat([W_features.data,W_bias.data, M_features.data, M_bias.data],axis=1))
df_ebdspace.columns = ['여성잠재특징1','여성잠재특징2','여성바이어스','남성잠재특징1','남성잠재특징2','남성바이어스']
df_yhat = pd.DataFrame(yhat.data,columns=['yhat'])
pd.concat([df,df_yhat,df_ebdspace],axis=1)

,W,M,y,yhat,여성잠재특징1,여성잠재특징2,여성바이어스,남성잠재특징1,남성잠재특징2,남성바이어스
0,옥순,영식,3.9,3.937361,-0.995794,-1.021227,-0.037432,-1.237555,-0.286875,-0.070347
1,옥순,영철,4.1,4.075119,-0.995794,-1.021227,-0.037432,-1.579593,-0.060340,-0.038357
2,옥순,광수,0.5,0.474808,-0.995794,-1.021227,-0.037432,0.643061,1.397840,-0.055886
3,옥순,상철,0.3,0.325054,-0.995794,-1.021227,-0.037432,0.816490,1.898078,0.080153
4,영자,영식,4.5,4.439692,-1.527894,-0.829441,0.040873,-1.237555,-0.286875,-0.070347
5,영자,영호,3.7,3.728190,-1.527894,-0.829441,0.040873,-1.494347,1.103693,-0.187010
6,영자,광수,0.5,0.511475,-1.527894,-0.829441,0.040873,0.643061,1.397840,-0.055886
7,영자,영수,0.4,0.371204,-1.527894,-0.829441,0.040873,1.027473,1.316166,0.028253
8,정숙,영철,4.9,4.901733,-1.504516,-0.851983,0.779223,-1.579593,-0.060340,-0.038357
9,정숙,영호,4.7,4.618024,-1.504516,-0.851983,0.779223,-1.494347,1.103693,-0.187010


# 6. 나는 SOLO 다른구현 -- 임베딩레이어

# 7. 나는 SOLO 다른구현 -- net 이용 

# 8. 나는 SOLO 다른구현 -- Fastai

In [314]:
dls = fastai.collab.CollabDataLoaders.from_df(df,bs=2,valid_pct = 2/24)
lrnr = fastai.collab.collab_learner(dls, n_factors=2, y_range=(0,5))
lrnr.fit(30,lr=0.1)

epoch,train_loss,valid_loss,time
0,4.062271,4.810657,00:00
1,3.748495,7.016310,00:00
2,3.368854,9.019559,00:00
3,2.922977,9.697640,00:00
4,2.488001,9.183447,00:00
5,2.199031,8.115675,00:00
6,1.905716,7.810968,00:00
7,1.657743,7.658912,00:00
8,1.407650,7.600761,00:00
9,1.219581,7.793707,00:00
